# Imports

In [1]:
import pandas as pd
import inflection 

# Funções

In [2]:
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

def dict_to_df(dicto, col1, col2):
    df = pd.DataFrame.from_dict(dicto, orient='index')
    df = df.reset_index()
    df.rename(columns = {'index':col1, 0: col2}, inplace=True)
    return df

def transform(data, countries, colors, fx):
    ## Removendo Outlier da Austrália 
    cond1 = (data['country_code'] == 14)
    cond2 = (data['average_cost_for_two'] == 25000017)
    data.loc[cond1 & cond2, :] ## Verificando e confirmando de que é só australia
    data = data[data['average_cost_for_two'] != 25000017] # removendo da Base
    # Removendo NAN
    data = data.loc[pd.isna(data['cuisines']) != True, :]
    # Deixando Cuisines Unicos 
    data['cuisines'] = data.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])       
    # Removendo Duplicatas
    df = data.drop_duplicates(subset=['restaurant_id'])
    df = df.reset_index(drop=True)
    # Fazendo Merge com os DFs Colors e countries
    df1 = df.merge(countries, how='left', left_on='country_code', right_on='id')
    df1 = df1.merge(colors, how='left', left_on='rating_color', right_on='id')
    df1 = df1.merge(fx, how='left', left_on='currency', right_on='id')
    ### Converter Valores Para USD 
    df1['average_cost_for_two_usd'] = df1['average_cost_for_two'] * df1['price']
    # removendo NAN 
    df1 = df1.loc[df1['cuisines'] != 'nan', :]
    df1 = df1.reset_index(drop=True)
    # Removendo colunas 
    df1 = df1.drop(['country_code', 'id_x', 'id_y', 'id', 'price'], axis = 1)
    return df1

def price_range(price):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

# Extração

In [3]:
data = pd.read_csv('zomato.csv')

COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

FX = {
    'Botswana Pula(P)' : 0.076,
    'Brazilian Real(R$)': 0.19,
    'Dollar($)': 1,
    'Emirati Diram(AED)': 0.27,
    'Indian Rupees(Rs.)': 0.012,
    'Indonesian Rupiah(IDR)': 0.000064,
    'NewZealand($)' : 0.64,
    'Pounds(£)': 1.21,
    'Qatari Rial(QR)' : 0.27,
    'Rand(R)' : 0.057,
    'Sri Lankan Rupee(LKR)': 0.027,
    'Turkish Lira(TL)': 0.054
    
}

# Limpando e Criando Colunas

In [4]:
colors = dict_to_df(COLORS, 'id', 'color')


In [5]:
countries = dict_to_df(COUNTRIES, 'id', 'country')


In [6]:
fx = dict_to_df(FX, 'id', 'price')

In [7]:
# Mudando o nome das Colunas e 
data = rename_columns(data)

In [8]:
df1 = transform(data, countries, colors, fx)

# Dados Limpos

In [9]:
data_raw = df1.copy()

# Analisando a base

In [10]:
#========================================================#
                   # Análise Geral #
#========================================================#


# 1.Quantos restaurantes únicos estão registrados?
qtd_restaurante = len(data_raw['restaurant_id'].unique())
print(qtd_restaurante)

# 2.Quantos países únicos estão registrados?
qts_countries = len(data_raw['country'].unique())
print(qts_countries)

# 3.Quantas cidades únicas estão registradas?
qtd_city = len(data_raw['city'].unique())
print(qtd_city)

# 4.Qual o total de avaliações feitas?
total_votes = data_raw['votes'].sum()
print(total_votes)

# 5.Qual o total de tipos de culinária registrados?
cuisines_type = len(data_raw['cuisines'].unique())

6928
15
125
4194330


In [16]:
#========================================================#
                   # Pais #
#========================================================#

#1. Qual o nome do país que possui mais cidades registradas?
pa1 = data_raw[['country', 'city']].groupby('country').nunique().reset_index()
pa1 = pa1.sort_values('city', ascending = False).reset_index(drop=True)

#2. Qual o nome do país que possui mais restaurantes registrados?
pa2 = data_raw[['country', 'restaurant_id']].groupby('country').count().reset_index().sort_values('restaurant_id', ascending = False)

#3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?
linhas = data_raw['price_range'] == 4
pa3 = data_raw.loc[linhas, :][['country', 'restaurant_id']].groupby('country').count().reset_index().sort_values('restaurant_id', ascending = False)

#4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?
pa4 = data_raw[['country', 'cuisines']].groupby('country').nunique()

#5. Qual o nome do país que possui a maior quantidade de avaliações feitas?
pa5 = data_raw[['country', 'votes']].groupby('country').sum().reset_index().sort_values('votes', ascending = False)

#6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?
linhas1 = data_raw['has_online_delivery'] == 1 
pa6 = data_raw.loc[linhas1, :][['country', 'restaurant_id']].groupby('country').count().reset_index().sort_values('restaurant_id', ascending = False)

#7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?
linhas2 = data_raw['has_table_booking'] == 1 
pa7 = data_raw.loc[linhas2, :][['country', 'restaurant_id']].groupby('country').count().reset_index().sort_values('restaurant_id', ascending = False)

#8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?
pa8 = data_raw[['country', 'votes']].groupby('country').mean().reset_index().sort_values('votes', ascending = False)
pa8['votes'] = round(pa8['votes'], 2)

#9. Qual o nome do país que possui, na média, a maior nota média registrada?
pa9 = data_raw[['country', 'aggregate_rating']].groupby('country').max().reset_index().sort_values('aggregate_rating', ascending = False)

#10. Qual o nome do país que possui, na média, a menor nota média registrada?
pa10 = data_raw[['country', 'aggregate_rating']].groupby('country').min().reset_index().sort_values('aggregate_rating', ascending = True)

#11. Qual a média de preço de um prato para dois por país?
pa11 = data_raw[['country', 'average_cost_for_two_usd']].groupby('country').mean().reset_index().sort_values('average_cost_for_two_usd', ascending = False)

In [12]:
#========================================================#
                   # Cidade #
#========================================================#

#1. Qual o nome da cidade que possui mais restaurantes registrados?
c1 = data_raw[['city', 'restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending = False)

#2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?
c2 = data_raw.loc[data_raw['aggregate_rating'] >= 4.0, :][['city', 'restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending = False)

#3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?
c3 = data_raw.loc[data_raw['aggregate_rating'] < 2.5, :][['city', 'restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending = False)

#4. Qual o nome da cidade que possui o maior valor médio de um prato para dois? 
c4 = data_raw[['city', 'average_cost_for_two_usd']].groupby('city').max().reset_index().sort_values('average_cost_for_two_usd', ascending = False)

#5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?
c5 = data_raw[['city', 'cuisines']].groupby('city').nunique().reset_index().sort_values('cuisines', ascending = False)

#6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?
c6 = data_raw.loc[data_raw['has_table_booking'] == 1, :][['city', 'restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending = False)

#7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?
c7 = data_raw.loc[data_raw['is_delivering_now'] == 1, :][['city', 'restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending = False)

#8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?
c8 = data_raw.loc[data_raw['has_online_delivery'] == 1, :][['city', 'restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending = False)


In [13]:
#========================================================#
                   # Culinária #
#========================================================#

#1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?
c1 = data_raw.loc[data_raw['cuisines'] =='Italian', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                            ascending=False).reset_index()   
#2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?
c2= data_raw.loc[data_raw['cuisines'] =='Italian', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                            ascending=True).reset_index()
#3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?
c3 = data_raw.loc[data_raw['cuisines'] =='American', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                        ascending=False).reset_index()
#4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?
c3 = data_raw.loc[data_raw['cuisines'] =='American', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                        ascending=True).reset_index()
#5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?
c5 = data_raw.loc[data_raw['cuisines'] =='Arabian', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                        ascending=False).reset_index()
#6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?
c6 = data_raw.loc[data_raw['cuisines'] =='Arabian', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                        ascending=True).reset_index()
#7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?
c7 = data_raw.loc[data_raw['cuisines'] =='Japanese', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                        ascending=False).reset_index()
#8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?
c8 =data_raw.loc[data_raw['cuisines'] =='Japanese', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                        ascending=True).reset_index()
#9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?
c9 = data_raw.loc[data_raw['cuisines'] =='Home-made', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                        ascending=False).reset_index()
#10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?
c10 = data_raw.loc[data_raw['cuisines'] =='Home-made', :][['restaurant_name', 'aggregate_rating']].groupby('restaurant_name').max().sort_values('aggregate_rating',
                                                                                                                                        ascending=True).reset_index()
#11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?
c11 = data_raw[['cuisines','average_cost_for_two_usd']].groupby('cuisines').max().sort_values('average_cost_for_two_usd', ascending = False).reset_index()

#12. Qual o tipo de culinária que possui a maior nota média?
c12 = data_raw[['cuisines','aggregate_rating']].groupby('cuisines').max().sort_values('aggregate_rating', ascending = False).reset_index()

#13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas
c13a = data_raw.loc[(data_raw['has_online_delivery'] == 1) & (data_raw['is_delivering_now'] == 1) , :][['cuisines', 'restaurant_id' ]]
c13 = c13a.groupby('cuisines').count().sort_values('restaurant_id', ascending = False).reset_index()

In [25]:
pa11.iloc[0:5, :]

,country,average_cost_for_two_usd
9,Singapure,141.437500
7,Philippines,93.314700
0,Australia,71.195531
11,Sri Lanka,69.643125
14,United States of America,55.029840
